In [12]:
import torch
import torchvision
import torchvision.transforms as transforms
import torch.nn as nn
import torch.optim as optim
import numpy as np
from torchvision.models import mobilenet_v2

In [13]:
# 转换器
transform = transforms.Compose([
    transforms.ToTensor(),
    transforms.Normalize((0.5, 0.5, 0.5), (0.5, 0.5, 0.5))
])

# 加载训练集和测试集
trainset = torchvision.datasets.CIFAR10(root='./data', train=True,
                                        download=False, transform=transform)
testset = torchvision.datasets.CIFAR10(root='./data', train=False,
                                       download=False, transform=transform)

trainloader = torch.utils.data.DataLoader(trainset, batch_size=64,
                                          shuffle=True, num_workers=2)
testloader = torch.utils.data.DataLoader(testset, batch_size=64,
                                         shuffle=False, num_workers=2)


In [14]:
# 添加标签噪声的函数
def add_label_noise(labels, noise_type='symmetric', noise_rate=0.2):
    noisy_labels = labels.copy()
    n_samples = labels.shape[0]
    n_classes = 10

    if noise_type == 'symmetric':
        n_noisy = int(noise_rate * n_samples)
        noise_idx = np.random.choice(n_samples, n_noisy, replace=False)
        for idx in noise_idx:
            original_label = labels[idx]
            noisy_labels[idx] = np.random.choice([l for l in range(n_classes) if l != original_label])
    elif noise_type == 'pairflip':
        for i in range(n_classes // 2):
            flip_from = i * 2
            flip_to = flip_from + 1
            flip_idx = np.where(labels == flip_from)[0]
            n_flip = int(noise_rate * len(flip_idx))
            flip_idx = np.random.choice(flip_idx, n_flip, replace=False)
            noisy_labels[flip_idx] = flip_to

    return noisy_labels

In [15]:
device = torch.device("mps" if torch.backends.mps.is_available() else "cpu")
print("Using device:", device)

Using device: mps


In [17]:
# MobileNet 模型
net = mobilenet_v2(pretrained=False)
net.classifier[1] = nn.Linear(net.classifier[1].in_features, 10)
net.to(device)
print("start")

# 训练模型
criterion = nn.CrossEntropyLoss()
optimizer = optim.SGD(net.parameters(), lr=0.003, momentum=0.9)

for epoch in range(20):  # 遍历数据集多次
    running_loss = 0.0
    for i, data in enumerate(trainloader, 0):
        inputs, labels = data
        inputs, labels = inputs.to(device), labels.to(device)
        #labels = labels.numpy()
        labels = labels.cpu().numpy()
        # 添加噪声
        noisy_labels = add_label_noise(labels, noise_type='symmetric', noise_rate=0.4)
        noisy_labels = torch.from_numpy(noisy_labels).to(device)

        optimizer.zero_grad()

        outputs = net(inputs)
        loss = criterion(outputs, noisy_labels.long())
        loss.backward()
        optimizer.step()

        running_loss += loss.item()
         # 打印当前进度
        if i % 200 == 199:    # 每 200 批次打印一次
            print(f'Epoch: {epoch + 1}, Batch: {i + 1}/{len(trainloader)}, Loss: {running_loss / 200}')
            running_loss = 0.0
            

    print(f'Epoch {epoch + 1} loss: {running_loss / len(trainloader)}')

print('Finished Training')

# 评估模型
correct = 0
total = 0
with torch.no_grad():
    for data in testloader:
        images, labels = data
        images, labels = images.to(device), labels.to(device)
        outputs = net(images)
        _, predicted = torch.max(outputs.data, 1)
        total += labels.size(0)
        correct += (predicted == labels).sum().item()

print(f'Accuracy of the network on the 10000 test images: {100 * correct // total} %')

start
Epoch: 1, Batch: 200/782, Loss: 2.350317219495773
Epoch: 1, Batch: 400/782, Loss: 2.313448450565338
Epoch: 1, Batch: 600/782, Loss: 2.2756838822364807
Epoch 1 loss: 0.5277622441196685
Epoch: 2, Batch: 200/782, Loss: 2.2473407316207887
Epoch: 2, Batch: 400/782, Loss: 2.2372902154922487
Epoch: 2, Batch: 600/782, Loss: 2.224487934112549
Epoch 2 loss: 0.5156952391195175
Epoch: 3, Batch: 200/782, Loss: 2.197950953841209
Epoch: 3, Batch: 400/782, Loss: 2.1988168001174926
Epoch: 3, Batch: 600/782, Loss: 2.1841764998435975
Epoch 3 loss: 0.5050910580188722
Epoch: 4, Batch: 200/782, Loss: 2.1532162684202194
Epoch: 4, Batch: 400/782, Loss: 2.1637836158275605
Epoch: 4, Batch: 600/782, Loss: 2.152998569011688
Epoch 4 loss: 0.4993954826803768
Epoch: 5, Batch: 200/782, Loss: 2.133040486574173
Epoch: 5, Batch: 400/782, Loss: 2.1288128554821015
Epoch: 5, Batch: 600/782, Loss: 2.1287280148267747
Epoch 5 loss: 0.4952444347274273
Epoch: 6, Batch: 200/782, Loss: 2.108162282705307
Epoch: 6, Batch: 400